In [1]:
import pandas as pd
pd.set_option("display.max_colwidth", 200)
import numpy as np
import json
import re
import nltk
from nltk import FreqDist
import spacy

import gensim
from gensim import corpora

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\saksh\Anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [2]:
df = pd.read_csv('Tweets for top 20 influencers.csv')  

In [3]:
df.head()

,index,Screen Name,Date,Tweet,Likes,Comments,Retweets,Mentions,Clean Text,Id,Name,Location,Followers Count,Friends Count,Listed Count,Created At,Favourites Count,Verified,Tweets Count
0,0,donwinslow,27-Nov,Dear @realDonaldTrump President-elect Biden's national popular vote lead climbs to 6.28 million.,15800,173,2300,@realDonaldTrump,dear president elect national popular vote lead climb million,255812611.0,Don Winslow,Repped by The Story Factory,540001,18502,2579,2/22/2011 2:58,168471,True,59151
1,81,donwinslow,27-Nov,Dear @realDonaldTrump This is incredibly simple. You lost.,16300,325,1900,@realDonaldTrump,dear incredibly simple lost,255812611.0,Don Winslow,Repped by The Story Factory,540001,18502,2579,2/22/2011 2:58,168471,True,59151
2,94,donwinslow,26-Nov,"Dear @realDonaldTrump It's going to be a very different ballgame after January 20, 2021.",9600,172,935,@realDonaldTrump,dear going different,255812611.0,Don Winslow,Repped by The Story Factory,540001,18502,2579,2/22/2011 2:58,168471,True,59151
3,151,donwinslow,26-Nov,I'm thankful to all the people who didn't travel for Thanksgiving and I'm thankful Donald Trump will never be in power again.,62100,813,4300,NaN,thankful people travel thanksgiving thankful trump never power,255812611.0,Don Winslow,Repped by The Story Factory,540001,18502,2579,2/22/2011 2:58,168471,True,59151
4,234,donwinslow,25-Nov,What James Comey said about Michael Flynn was all true. What Barack Obama told Donald Trump about Michael Flynn was all true. Michael Flynn broke the law. Donald Trump broke the law. There wil...,9500,213,1500,NaN,said true told trump true broke law trump broke law pardon,255812611.0,Don Winslow,Repped by The Story Factory,540001,18502,2579,2/22/2011 2:58,168471,True,59151


In [3]:
a=df['Tweets Count'].unique()

In [4]:
a.sort()
a

array([  3384,   4276,   4637,   6869,  11693,  12341,  12724,  12837,
        16815,  19688,  21677,  23641,  27541,  30313,  40485,  44452,
        46231,  59151,  82193, 325647], dtype=int64)

In [8]:
def lemmatization(texts, tags=['NOUN', 'ADJ']): # filter noun and adjective
    output = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        output.append([token.lemma_ for token in doc if token.pos_ in tags])
    return output

# MURRAY🧢🇺🇸

In [9]:
df_max_tc = df[df['Screen Name']== "tedlieu"]
tokenized_reviews = pd.Series(df_max_tc['Clean Text'].tolist()).apply(lambda x: x.split())
nlp = spacy.load('en', disable=['parser', 'ner'])
reviews_2 = lemmatization(tokenized_reviews)
remove_list = ['trump','#','']
for i in range(len(reviews_2)):
    append_list = []
    for j in range(len(reviews_2[i])):
        if reviews_2[i][j] not in remove_list:
            append_list.append(reviews_2[i][j])
    reviews_2[i]=append_list
dictionary = corpora.Dictionary(reviews_2)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in reviews_2]
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=5, random_state=100,
                chunksize=1000, passes=50)
lda_model.print_topics()
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

C:\Users\saksh\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.130224  0.004603       1        1  25.732475
2     -0.057203 -0.111320       2        1  25.398333
4     -0.026978  0.013718       3        1  18.402624
0      0.016412 -0.010003       4        1  15.564153
1     -0.062456  0.103001       5        1  14.902423, topic_info=    Category       Freq       Term      Total  loglift  logprob
498  Default  11.000000  president  11.000000  30.0000  30.0000
44   Default  11.000000   stimulus  11.000000  29.0000  29.0000
200  Default   9.000000     speech   9.000000  28.0000  28.0000
5    Default  26.000000   election  26.000000  27.0000  27.0000
76   Default  27.000000       vote  27.000000  26.0000  26.0000
..       ...        ...        ...        ...      ...      ...
62    Topic5   1.441272      point   6.951601   0.3302  -5.4866
369   Topic5   1.440400       life   9.281755   0.0405  -5.4872
357   Topic5   1.439908     health   4.133899   0.8490  -5.4875
94    Topic5   1.439441   possible   5.668842   0.5329  -5.4879
229   Topic5   1.439358        man   4.978909   0.6626  -5.4879

[326 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
446       1  0.693119    account
446       4  0.231040    account
29        3  0.960150     active
406       4  0.491864   approach
147       4  0.731973  ascertain
...     ...       ...        ...
405       2  0.164410       year
405       3  0.164410       year
405       4  0.164410       year
405       5  0.082205       year
136       1  0.915636      youth

[469 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 5, 1, 2])

In [8]:
print("Number of Tweet: ",len(tokenized_reviews))
print("Tweet count", a[-11])

Number of Tweet:  88
Tweet count 24425


# Don Winslow

In [9]:
df_max_tc = df[df['Tweets Count']==a[-7]]
tokenized_reviews = pd.Series(df_max_tc['Clean Text'].tolist()).apply(lambda x: x.split())
nlp = spacy.load('en', disable=['parser', 'ner'])
reviews_2 = lemmatization(tokenized_reviews)
remove_list = ['trump','#','']
for i in range(len(reviews_2)):
    append_list = []
    for j in range(len(reviews_2[i])):
        if reviews_2[i][j] not in remove_list:
            append_list.append(reviews_2[i][j])
    reviews_2[i]=append_list
dictionary = corpora.Dictionary(reviews_2)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in reviews_2]
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=6, random_state=100,
                chunksize=1000, passes=50)
lda_model.print_topics()
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.133842 -0.008623       1        1  23.789255
5      0.035167 -0.077113       2        1  19.642691
1     -0.046785 -0.004302       3        1  14.799209
4      0.018609  0.142638       4        1  14.392532
3      0.032896 -0.059683       5        1  14.290528
0      0.093955  0.007083       6        1  13.085793, topic_info=         Term       Freq      Total Category  logprob  loglift
24    country  11.000000  11.000000  Default  30.0000  30.0000
99        new  13.000000  13.000000  Default  29.0000  29.0000
93   criminal   4.000000   4.000000  Default  28.0000  28.0000
536    parent   4.000000   4.000000  Default  27.0000  27.0000
0        dear  19.000000  19.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
168      life   1.348117   5.314145   Topic6  -5.0416   0.6620
264      view   1.348799   6.692718   Topic6  -5.0411   0.4318
64   election   1.345946   8.050486   Topic6  -5.0433   0.2450
5      people   1.347746  18.650944   Topic6  -5.0419  -0.5938
33       link   1.346162  20.148075   Topic6  -5.0431  -0.6722

[364 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
557       3  0.762128    abuse
279       3  0.225052  account
279       4  0.225052  account
279       6  0.675157  account
697       5  0.764573      act
...     ...       ...      ...
91        1  0.305831     year
91        2  0.305831     year
91        3  0.229373     year
91        5  0.076458     year
433       4  0.766090   zoning

[479 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 6, 2, 5, 4, 1])

In [10]:
print("Number of Tweet: ",len(tokenized_reviews))
print("Tweet count", a[-7])

Number of Tweet:  235
Tweet count 59151


# George Conway

In [11]:
df_max_tc = df[df['Tweets Count']==a[-6]]
tokenized_reviews = pd.Series(df_max_tc['Clean Text'].tolist()).apply(lambda x: x.split())
nlp = spacy.load('en', disable=['parser', 'ner'])
reviews_2 = lemmatization(tokenized_reviews)
remove_list = ['trump','#','']
for i in range(len(reviews_2)):
    append_list = []
    for j in range(len(reviews_2[i])):
        if reviews_2[i][j] not in remove_list:
            append_list.append(reviews_2[i][j])
    reviews_2[i]=append_list
dictionary = corpora.Dictionary(reviews_2)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in reviews_2]
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=7, random_state=100,
                chunksize=1000, passes=50)
lda_model.print_topics()
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.104621 -0.101220       1        1  17.038733
0      0.146921  0.008867       2        1  17.008001
4      0.035845 -0.019802       3        1  15.596837
5      0.036274 -0.084940       4        1  14.037726
3     -0.008823  0.036423       5        1  12.196112
2     -0.063439  0.047309       6        1  12.069180
1     -0.042157  0.113363       7        1  12.053405, topic_info=       Term      Freq     Total Category  logprob  loglift
62   people  7.000000  7.000000  Default  30.0000  30.0000
287     tax  3.000000  3.000000  Default  29.0000  29.0000
268   first  5.000000  5.000000  Default  28.0000  28.0000
37     year  6.000000  6.000000  Default  27.0000  27.0000
112    vote  5.000000  5.000000  Default  26.0000  26.0000
..      ...       ...       ...      ...      ...      ...
126    real  0.669096  3.049701   Topic7  -5.0705   0.5990
454   close  0.669091  1.867851   Topic7  -5.0705   1.0892
165    late  0.669088  1.861662   Topic7  -5.0705   1.0925
262  matter  0.669087  1.852352   Topic7  -5.0705   1.0975
314  police  0.669084  3.040458   Topic7  -5.0705   0.6020

[412 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
49        3  0.789324  abdication
50        3  0.789324      abject
105       1  0.517831    absentee
64        4  0.546169     account
64        5  0.546169     account
...     ...       ...         ...
37        3  0.149560        year
37        6  0.149560        year
37        7  0.448680        year
176       2  0.525523   yesterday
176       4  0.525523   yesterday

[494 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 1, 5, 6, 4, 3, 2])

In [12]:
print("Number of Tweet: ",len(tokenized_reviews))
print("Tweet count", a[-6])

Number of Tweet:  105
Tweet count 73478


# Make Kindness Cool Again

In [13]:
df_max_tc = df[df['Tweets Count']==a[-5]]
tokenized_reviews = pd.Series(df_max_tc['Clean Text'].tolist()).apply(lambda x: x.split())
nlp = spacy.load('en', disable=['parser', 'ner'])
reviews_2 = lemmatization(tokenized_reviews)
remove_list = ['trump','#','']
for i in range(len(reviews_2)):
    append_list = []
    for j in range(len(reviews_2[i])):
        if reviews_2[i][j] not in remove_list:
            append_list.append(reviews_2[i][j])
    reviews_2[i]=append_list
dictionary = corpora.Dictionary(reviews_2)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in reviews_2]
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=6, random_state=100,
                chunksize=1000, passes=50)
lda_model.print_topics()
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.170461  0.089383       1        1  21.636356
5     -0.078601 -0.125181       2        1  17.674055
4      0.020361 -0.001439       3        1  16.041836
2      0.080616  0.057217       4        1  15.783710
0      0.050453 -0.058595       5        1  14.449498
1      0.097633  0.038615       6        1  14.414541, topic_info=           Term       Freq      Total Category  logprob  loglift
3          vote  37.000000  37.000000  Default  30.0000  30.0000
1         legal  19.000000  19.000000  Default  29.0000  29.0000
28         noon   7.000000   7.000000  Default  28.0000  28.0000
94          day  11.000000  11.000000  Default  27.0000  27.0000
38        audit   6.000000   6.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
113        long   0.753699   2.617039   Topic6  -5.3268   0.6921
122  leadership   0.753635   1.961906   Topic6  -5.3269   0.9802
303        fake   0.753552   2.642039   Topic6  -5.3270   0.6824
212     violent   0.753503   2.629414   Topic6  -5.3271   0.6871
337      prison   0.753460   1.974315   Topic6  -5.3272   0.9736

[358 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
147       5  0.766464    absentee
311       3  0.500978     account
49        2  0.497758  additional
16        2  0.300738     address
16        3  0.300738     address
...     ...       ...         ...
31        1  0.373042        year
31        3  0.497389        year
31        4  0.124347        year
32        3  0.381661   yesterday
32        6  0.381661   yesterday

[434 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 5, 3, 1, 2])

In [14]:
print("Number of Tweet: ",len(tokenized_reviews))
print("Tweet count", a[-5])

Number of Tweet:  165
Tweet count 82193


# Hoodlum

In [15]:
df_max_tc = df[df['Tweets Count']==a[-4]]
tokenized_reviews = pd.Series(df_max_tc['Clean Text'].tolist()).apply(lambda x: x.split())
nlp = spacy.load('en', disable=['parser', 'ner'])
reviews_2 = lemmatization(tokenized_reviews)
remove_list = ['trump','#','']
for i in range(len(reviews_2)):
    append_list = []
    for j in range(len(reviews_2[i])):
        if reviews_2[i][j] not in remove_list:
            append_list.append(reviews_2[i][j])
    reviews_2[i]=append_list
dictionary = corpora.Dictionary(reviews_2)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in reviews_2]
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=4, random_state=100,
                chunksize=1000, passes=50)
lda_model.print_topics()
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.143658 -0.017464       1        1  70.888367
3     -0.103494 -0.065007       2        1  15.541087
0     -0.046033  0.090613       3        1  11.354168
1      0.005869 -0.008142       4        1   2.216374, topic_info=            Term     Freq     Total Category  logprob  loglift
5   presidential  0.00000  0.000000  Default  23.0000  23.0000
6        recount  0.00000  0.000000  Default  22.0000  22.0000
7         reject  0.00000  0.000000  Default  21.0000  21.0000
8        waiting  0.00000  0.000000  Default  20.0000  20.0000
9           year  0.00000  0.000000  Default  19.0000  19.0000
..           ...      ...       ...      ...      ...      ...
17    protection  0.03372  2.066799   Topic4  -3.1357  -0.3064
18          rare  0.03372  2.066799   Topic4  -3.1357  -0.3064
20       service  0.03372  2.066799   Topic4  -3.1357  -0.3064
21         sheep  0.03372  2.873694   Topic4  -3.1357  -0.6360
22          trip  0.03372  2.873695   Topic4  -3.1357  -0.6360

[115 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
0         2  1.091662    allegation
1         2  1.091662        appeal
10        1  0.967680         breed
2         2  1.091662        charge
11        1  0.967680      document
12        1  0.967680         giant
13        1  0.793714    government
14        1  0.793714       hunting
15        1  0.967680          last
16        1  0.793714          link
5         3  1.161947  presidential
3         2  1.091662         proof
17        1  0.967680    protection
18        1  0.967680          rare
6         3  1.161947       recount
7         3  1.161947        reject
19        1  0.967680        secret
20        1  0.967680       service
21        1  1.043952         sheep
4         2  1.091662      specific
22        1  1.043952          trip
8         3  1.161947       waiting
9         2  1.091883          year, R=23, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 1, 2])

In [16]:
print("Number of Tweet: ",len(tokenized_reviews))
print("Tweet count", a[-4])

Number of Tweet:  4
Tweet count 87476


# Tony Posnanski

In [17]:
df_max_tc = df[df['Tweets Count']==a[-2]]
#df_max_tc.head()
tokenized_reviews = pd.Series(df_max_tc['Clean Text'].tolist()).apply(lambda x: x.split())
nlp = spacy.load('en', disable=['parser', 'ner'])
reviews_2 = lemmatization(tokenized_reviews)
remove_list = ['trump','#','']
for i in range(len(reviews_2)):
    append_list = []
    for j in range(len(reviews_2[i])):
        if reviews_2[i][j] not in remove_list:
            append_list.append(reviews_2[i][j])
    reviews_2[i]=append_list
dictionary = corpora.Dictionary(reviews_2)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in reviews_2]
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=4, random_state=100,
                chunksize=1000, passes=50)
lda_model.print_topics()
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.112612 -0.047003       1        1  39.433487
3     -0.104927 -0.057351       2        1  34.199749
1     -0.006682  0.109134       3        1  22.010164
0     -0.001003 -0.004780       4        1   4.356603, topic_info=           Term      Freq     Total Category  logprob  loglift
4          desk  0.000000  0.000000  Default  30.0000  30.0000
5        school  0.000000  0.000000  Default  29.0000  29.0000
30         work  1.000000  1.000000  Default  28.0000  28.0000
39      vaccine  2.000000  2.000000  Default  27.0000  27.0000
19        clear  0.000000  0.000000  Default  26.0000  26.0000
..          ...       ...       ...      ...      ...      ...
9          case  0.044164  1.108859   Topic4  -4.2904  -0.0897
14  partnership  0.044164  1.108859   Topic4  -4.2904  -0.0897
16         slow  0.044164  1.108859   Topic4  -4.2904  -0.0897
11     decrease  0.044164  1.108859   Topic4  -4.2904  -0.0897
49         well  0.044162  1.108864   Topic4  -4.2905  -0.0898

[181 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
8         1  0.901828    action
36        1  0.901853       aid
40        1  0.901824     black
51        2  0.928575      blue
41        1  0.901824  business
...     ...       ...       ...
63        2  0.928575     voter
64        2  0.928575       way
49        1  0.901824      well
30        3  0.672061      work
50        1  0.901824      year

[67 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 1])

In [18]:
print("Number of Tweet: ",len(tokenized_reviews))
print("Tweet count", a[-2])

Number of Tweet:  10
Tweet count 137829


# BrooklynDad_Defiant

In [19]:

df_max_tc = df[df['Tweets Count']==a[-3]]
tokenized_reviews = pd.Series(df_max_tc['Clean Text'].tolist()).apply(lambda x: x.split())
nlp = spacy.load('en', disable=['parser', 'ner'])
reviews_2 = lemmatization(tokenized_reviews)
remove_list = ['trump','#','']
for i in range(len(reviews_2)):
    append_list = []
    for j in range(len(reviews_2[i])):
        if reviews_2[i][j] not in remove_list:
            append_list.append(reviews_2[i][j])
    reviews_2[i]=append_list
dictionary = corpora.Dictionary(reviews_2)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in reviews_2]
# corpora.MmCorpus.serialize('XX.mm', doc_term_matrix)  # store to disk, for later use
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=6, random_state=100,
                chunksize=1000, passes=50)
#lda_model.print_topics()
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.103170 -0.128222       1        1  21.480593
3     -0.145157 -0.046843       2        1  21.034571
1     -0.026939 -0.008338       3        1  17.826262
4      0.040111  0.111421       4        1  13.517845
5      0.049869  0.045832       5        1  13.178189
2     -0.021052  0.026149       6        1  12.962543, topic_info=              Term       Freq      Total Category  logprob  loglift
37          people  10.000000  10.000000  Default  30.0000  30.0000
204            day   4.000000   4.000000  Default  29.0000  29.0000
151          money   2.000000   2.000000  Default  28.0000  28.0000
462       involved   2.000000   2.000000  Default  27.0000  27.0000
223           link   9.000000   9.000000  Default  26.0000  26.0000
..             ...        ...        ...      ...      ...      ...
251  irresponsible   0.662965   1.827546   Topic6  -5.1399   1.0291
378  investigation   0.662962   1.845060   Topic6  -5.1399   1.0196
127        foreign   0.662955   1.845061   Topic6  -5.1399   1.0195
42           human   0.662920   1.771512   Topic6  -5.1399   1.0602
270           safe   0.662912   1.827555   Topic6  -5.1399   1.0290

[339 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
351       3  0.794061     absentee
310       1  0.384398      account
310       2  0.384398      account
498       3  0.794061  affiliation
7         4  0.558692          air
...     ...       ...          ...
333       2  0.516256        world
1         2  0.234459         year
1         4  0.468918         year
1         6  0.234459         year
460       5  0.835240         zone

[406 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 5, 6, 3])

In [20]:
print("Number of Tweet: ",len(tokenized_reviews))
print("Tweet count", a[-3])

Number of Tweet:  141
Tweet count 103622


# Patrick Chovanec

In [24]:

df_max_tc = df[df['Tweets Count']==a[-1]]
tokenized_reviews = pd.Series(df_max_tc['Clean Text'].tolist()).apply(lambda x: x.split())
nlp = spacy.load('en', disable=['parser', 'ner'])
reviews_2 = lemmatization(tokenized_reviews)
remove_list = ['trump','#','']
for i in range(len(reviews_2)):
    append_list = []
    for j in range(len(reviews_2[i])):
        if reviews_2[i][j] not in remove_list:
            append_list.append(reviews_2[i][j])
    reviews_2[i]=append_list
dictionary = corpora.Dictionary(reviews_2)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in reviews_2]
# corpora.MmCorpus.serialize('XX.mm', doc_term_matrix)  # store to disk, for later use
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=2, random_state=100,
                chunksize=1000, passes=50)
#lda_model.print_topics()
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=         x    y  topics  cluster       Freq
topic                                      
0     -0.0  0.0       1        1  95.061470
1      0.0  0.0       2        1   4.938533, topic_info=             Term      Freq  Total Category  logprob  loglift
1      compelling  0.000000    0.0  Default  11.0000     11.0
9         reality  0.000000    0.0  Default  10.0000     10.0
4          people  0.000000    0.0  Default   9.0000      9.0
0      attractive  0.000000    0.0  Default   8.0000      8.0
8   psychological  1.000000    1.0  Default   7.0000      7.0
2       different  0.000000    0.0  Default   6.0000      6.0
6         politic  0.000000    0.0  Default   5.0000      5.0
3            full  1.000000    1.0  Default   4.0000      4.0
5           point  1.000000    1.0  Default   3.0000      3.0
7           prism  1.000000    1.0  Default   2.0000      2.0
10           view  1.000000    1.0  Default   1.0000      1.0
10           view  0.950615    1.0   Topic1  -2.3979      0.0
7           prism  0.950615    1.0   Topic1  -2.3979      0.0
9         reality  0.950614    1.0   Topic1  -2.3979      0.0
8   psychological  0.950615    1.0   Topic1  -2.3979      0.0
6         politic  0.950615    1.0   Topic1  -2.3979      0.0
5           point  0.950615    1.0   Topic1  -2.3979      0.0
4          people  0.950615    1.0   Topic1  -2.3979      0.0
3            full  0.950615    1.0   Topic1  -2.3979      0.0
2       different  0.950615    1.0   Topic1  -2.3979      0.0
1      compelling  0.950614    1.0   Topic1  -2.3979      0.0
0      attractive  0.950615    1.0   Topic1  -2.3979      0.0
9         reality  0.049385    1.0   Topic2  -2.3979      0.0
1      compelling  0.049385    1.0   Topic2  -2.3979      0.0
4          people  0.049385    1.0   Topic2  -2.3979      0.0
0      attractive  0.049385    1.0   Topic2  -2.3979      0.0
6         politic  0.049385    1.0   Topic2  -2.3979      0.0
2       different  0.049385    1.0   Topic2  -2.3979      0.0
8   psychological  0.049385    1.0   Topic2  -2.3979      0.0
3            full  0.049385    1.0   Topic2  -2.3979     -0.0
5           point  0.049385    1.0   Topic2  -2.3979     -0.0
7           prism  0.049385    1.0   Topic2  -2.3979     -0.0
10           view  0.049385    1.0   Topic2  -2.3979     -0.0, token_table=      Topic  Freq           Term
term                            
0         1   1.0     attractive
1         1   1.0     compelling
2         1   1.0      different
3         1   1.0           full
4         1   1.0         people
5         1   1.0          point
6         1   1.0        politic
7         1   1.0          prism
8         1   1.0  psychological
9         1   1.0        reality
10        1   1.0           view, R=11, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [25]:
print("Number of Tweet: ",len(tokenized_reviews))
print("Tweet count", a[-1])

Number of Tweet:  1
Tweet count 230189
